# Step 2 - Network analysis (R)

This R script will compute the travel time matrices between the first in the locations CSV and the remaining locations.  First run "Step 1 - Prepare input data.ipynb", then run this first to generate the matrices, then run "Step 3 - Final analysis.ipynb" to complete the analysis and generate the visualizations.

## Configure environment

Allocate memory to Java.  Currently set to allocate 8GB (hence the "8G" part of the string)

In [24]:
# Allocate RAM memory to Java
options(java.parameters = "-Xmx8G")
rJava::.jinit()
rJava::.jcall("java.lang.System", "S", "getProperty", "java.version")

[1] 0

[1] "11"

Install required R packages.  It is only necessary to run this one time.  Comment these out for subsequent runs to save time.

In [25]:
install.packages("r5r")
install.packages("sf")

Warning message:
"package 'r5r' is in use and will not be installed"
Warning message:
"package 'sf' is in use and will not be installed"


Load required packages into memory. This has to happen every time the script is run.

In [26]:
library(r5r)
library(sf)

## Define analysis parameters

In [27]:
# Default departure time for trips (you can override this below)
departureTime = as.POSIXct("2022-09-12 17:00:00",
                                 format = "%Y-%m-%d %H:%M:%S")

## Load contextual data

In [28]:
region <- st_read("./temp_data/region.shp")
region <- st_transform(region, crs="epsg:4326")

Reading layer `region' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\temp_data\region.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 12 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -84.01476 ymin: 39.16777 xmax: -82.02422 ymax: 40.71253
Geodetic CRS:  WGS 84


## Prepare transportation network

In [29]:
start_time = Sys.time()
r5r_core <- setup_r5(data_path = "./input_data", verbose=TRUE)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))


Using cached network.dat from ./input_data/network.dat



[1] "Compute time: 0.120659784475962 min"


## Prepare network endpoints

In [30]:
stop()

ERROR: Error in eval(expr, envir, enclos): 


The stop above was intentional. At this point, please run the included ArcPy script "r5prep.py", which will create "points.shp" and "tract_points.shp" (see below) by snapping the raw versions of the points (i.e. "points_raw.shp" and "tract_points_raw.shp" in the same directory) to the OSM network.  Alternatively, you could do this using a different tool such as QGIS or shapely.  You might even be able to simply copy (or rename) the raw files and use them as-is, although this might affect travel times and routing.

Load the location of the Intel plant.

In [31]:
site <- st_read("./input_data/site/site.shp")
site <- st_transform(site, crs="epsg:4326")

Reading layer `site' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\input_data\site\site.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 1 field
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -82.71037 ymin: 40.11924 xmax: -82.71037 ymax: 40.11924
Geodetic CRS:  WGS 84


Load the location of the New Albany COTA park and ride.

In [32]:
park_and_ride <- st_read("./temp_data/park_and_ride.shp")
park_and_ride <- st_transform(park_and_ride, crs="epsg:4326")

Reading layer `park_and_ride' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\temp_data\park_and_ride.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 1 field
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -82.78828 ymin: 40.0907 xmax: -82.78828 ymax: 40.0907
Geodetic CRS:  WGS 84


Load the high-resolution grid of points that will be used to construct isochrones. if you ran r5prep.py, these should be snapped to the street centerlines.

In [33]:
points <- st_read("./temp_data/points.shp")
points <- st_transform(points, crs="epsg:4326")

Reading layer `points' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\temp_data\points.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 7800 features and 1 field
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -84.01784 ymin: 39.16179 xmax: -82.03087 ymax: 40.71409
Geodetic CRS:  WGS 84


Load the census tract centroids, which are meant to represent destination neighborhoods. If you ran r5prep.py, these should be snapped to street centerlines.

In [34]:
tract_points <- st_read("./temp_data/tract_points.shp")
tract_points <- st_transform(tract_points, crs="epsg:4326")

Reading layer `tract_points' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\temp_data\tract_points.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 849 features and 1 field
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -84.15688 ymin: 39.00131 xmax: -81.97585 ymax: 40.82609
Geodetic CRS:  WGS 84


Load the approximate location of the Ohio GIS conference.

In [37]:
conference_location <- st_read("./temp_data/conference_location.shp")
conference_location <- st_transform(conference_location, crs="epsg:4326")

Reading layer `conference_location' from data source 
  `C:\Users\aporr\Desktop\Work\2022 Ohio GIS Conference\temp_data\conference_location.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 2 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -83.02748 ymin: 40.02204 xmax: -83.02748 ymax: 40.02204
Geodetic CRS:  WGS 84


## Compute travel time matrices from Ohio GIS conference to neighborhoods

Use snapped census tract centroids as a proxy for neighborhood centroids.

### By transit

In [38]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = conference_location,
                          destinations = tract_points,
                          mode = c("TRANSIT"),
                          departure_datetime = departureTime,
                          max_trip_duration = 90)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_TRANSIT_outbound_neighborhoods.csv", row.names=FALSE)

[1] "Compute time: 0.00174413124720256 min"


### By bike

In [39]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = conference_location,
                          destinations = tract_points,
                          mode = c("BICYCLE"),
                          departure_datetime = departureTime,
                          max_trip_duration = 90)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_BIKE_outbound_neighborhoods.csv", row.names=FALSE)

[1] "Compute time: 0.00642908414204915 min"


## Compute travel time matrices from Intel to regional grid points

Compute travel times to all points in dense grid covering the region.

### Evening weekday commute from Intel, car only, outbound, to whole region

In [16]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = site,
                          destinations = points,
                          mode = c("CAR"),
                          departure_datetime = departureTime,
                          max_trip_duration = 90)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_CAR_outbound_90.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.0907134334246318 min"


### Evening weekday commute from Intel, walk+transit, outbound, to whole region

In [17]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = site,
                          destinations = points,
                          mode = c("WALK", "TRANSIT"),
                          mode_egress = c("WALK"),
                          departure_datetime = departureTime,
                          max_trip_duration = 90)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_WALK-TRANSIT-WALK_outbound_90.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.0020597497622172 min"


### Evening weekday commute from Intel, bike+transit, outbound, to whole region

In [18]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = site,
                          destinations = points,
                          mode = c("BICYCLE","TRANSIT"),
                          mode_egress = c("BICYCLE"),
                          departure_datetime = departureTime,
                          max_trip_duration = 90)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_BIKE-TRANSIT-BIKE_outbound_90.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.00303986469904582 min"


### Evening weekday commute from Intel, shuttle+transit, outbound, to whole region

Actual origin point is the New Albany Park and Ride.  Ten minutes subtracted from max trip duration to account for implied shuttle from the plant to the park and ride. In theory R5 should be able to detect this route option on its own, but I could not get it to work. It's possible that deeper configuration of the R5 router is required.

In [19]:
start_time = Sys.time()
ttm <- travel_time_matrix(r5r_core = r5r_core,
                          origins = park_and_ride,
                          destinations = points,
                          mode = c("WALK","TRANSIT"),
                          mode_egress = c("WALK"),
                          departure_datetime = departureTime,
                          max_trip_duration = 80)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
write.csv(ttm, "output_data/ttm_SHUTTLE-TRANSIT-WALK_outbound_90.csv", row.names=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."
Warning message in assert_points_input(destinations, "destinations"):
"'destinations$id' forcefully cast to character."


[1] "Compute time: 0.00398684740066528 min"


## Compute detailed itineraries from Intel to Mariott Columbus OSU

In [ ]:
conference_location[1]

,id,geometry
,<chr>,<POINT [°]>
1,39049007820,POINT (-83.02748 40.02204)


In [43]:
start_time = Sys.time()
itineraries = detailed_itineraries(r5r_core = r5r_core,
  origins = site,
  destinations = conference_location[1],      # Tract 39049007820 centroid, near Mariott
  mode = c("TRANSIT", "WALK", "BICYCLE", "CAR"),
  mode_egress = c("WALK", "BICYCLE"),
  departure_datetime = departureTime,
  max_trip_duration = 150,
  shortest_path = FALSE
)
end_time = Sys.time()
duration = difftime(end_time, start_time, units="min")
print(paste0("Compute time: ", duration, " min"))
st_write(itineraries, "./output_data/itineraries_ANY_toconference_150.shp", append=FALSE)

Warning message in assert_points_input(origins, "origins"):
"'origins$id' forcefully cast to character."


[1] "Compute time: 0.00925701856613159 min"


Warning message in abbreviate_shapefile_names(obj):
"Field names abbreviated for ESRI Shapefile driver"


Deleting layer `itineraries_ANY_toconference_150' using driver `ESRI Shapefile'
Writing layer `itineraries_ANY_toconference_150' to data source 
  `./output_data/itineraries_ANY_toconference_150.shp' using driver `ESRI Shapefile'
Writing 13 features with 14 fields and geometry type Line String.


## Clean up memory

In [21]:
stop_r5(r5r_core)
rJava::.jgc(R.gc = TRUE)

r5r_core has been successfully stopped.



# References

  1. [r5r: Rapid Realistic Routing with R5 in R](https://ipeagit.github.io/r5r/) (Project website)
  1. [Intro to r5r: Rapid Realistic Routing with R5 in R](https://ipeagit.github.io/r5r/articles/r5r.html) (Vignette with multi-modal routing example using sample data)
  1. [r5r: Rapid Realistic Routing on Multimodal Transport Networks with R5 in R](https://findingspress.org/article/21262-r5r-rapid-realistic-routing-on-multimodal-transport-networks-with-r-5-in-r) (Published article with accessibility analysis example using sample data)
  1. [Calculating and visualizing Accessibility](https://ipeagit.github.io/r5r/articles/calculating_accessibility.html) (Vignette with accessibility analysis example using sample data)